In [ ]:
from sctoolbox.utilities import bgcolor

# Proportion Analysis
<hr style="border:2px solid black"> </hr>

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
%bgcolor PowderBlue

#Final clustering or celltype annotation column
clustering_col = "clustering"

#Set to None if not available
sample_col = "sample"

#Conditions to compare
condition_col = "chamber"
specific_conds = None  # specify conditions to compare: ["cond1", "cond2",...]. If None, all conditions are compared

#Transformation method
trans = 'logit'  # can be "logit" or "arcsin".

#Covariates to include in analysis
covariates = None

### For unreplicated data ###
#If sample_col=None, data is assumed unreplicated.
#Parameters for the bootstrapping if data is unreplicated
n_sims = 100  # number of bootstrapping simulations
n_reps = 8  # number of pseudo-replicates to generate for each condition

#P-value Threshold to determine significane
significance_threshold = 0.05

### Plots ###
#Clusters to plot
specific_clusters = None  # specify clusters you want to plot: ["c1", "c2",...], None to plot all
#Number of plots per row
n_cols = 4

<hr style="border:2px solid black"> </hr>

## Loading packages

In [ ]:
import logging
import scanpy as sc
import pandas as pd
pd.set_option('display.max_columns', None)  #no limit to the number of columns shown
import sctoolbox.utilities as utils
utils.settings_from_config("config.yaml", key="13")
from sctoolbox._settings import settings
import sctoolbox.utils.decorator as deco

#We will use Scanpro for proportion analysis
from scanpro import scanpro

## Loading adata

In [ ]:
adata = utils.load_h5ad("anndata_4.h5ad")
display(adata)

--------------

## Proportion analysis with Scanpro

In [ ]:
# add decorator to scanpro
scanpro = deco.log_anndata(scanpro)

In [ ]:
out = scanpro(adata,
              clusters_col=clustering_col,
              samples_col=sample_col,
              conds_col=condition_col,
              conditions=specific_conds,
              covariates=covariates,
              transform=trans,
              n_sims=n_sims,
              n_reps=n_reps)

out.results

In [ ]:
significant_change = (out.results['adjusted_p_values'] < significance_threshold).to_dict()
significant_change

In [ ]:
out.plot(kind='stripplot',
         clusters=specific_clusters,
         n_columns=n_cols,
         save=f'{settings.figure_dir}scanpro_stripplot.pdf')

In [ ]:
out.plot(kind='boxplot',
         clusters=specific_clusters,
         n_columns=n_cols,
         save=f'{settings.figure_dir}scanpro_boxplot.pdf')

In [ ]:
# Save results to uns dictionary
scanpro_uns_dict = {"scanpro": {"results": out.results,
                                "significance": significant_change,
                                "proportions": out.props,
                                "counts": out.counts,
                                "transformation": trans,
                                "conditions": out.conditions}}
# Add to adata
adata.uns.update(scanpro_uns_dict)

In [ ]:
utils.save_h5ad(adata, "anndata_13.h5ad")